<h1>Анализ базы данных приложения, предоставляющий сервис для чтения книг по подписке.

<big><strong>Описание проекта

Нам необходимо проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Цель исследования: анализ базы данных с целью сформулировать предложение для нового продукта

# ОГЛАВЛЕНИЕ

<a href=#1>Обзор данных.</a>

<a href=#2>Анализ данных.</a>
- <a href=#2.1>Сколько книг вышло после 1 января 2000 года.</a>
- <a href=#2.2>Количество обзоров и средняя оценка книг.</a>
- <a href=#2.3>Издательство, выпустившее наибольшее число книг.</a>
- <a href=#2.4>Автор с самой высокой средней оценкой книг.</a>
- <a href=#2.5>Среднее количество обзоров от пользователей, поставивших более 50 оценок.</a>

<a href=#3>Выводы.</a>



In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# <a name=1>Обзор данных.</a>

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [ ]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
# напишем функцию, выполняющую запрос на просмотр первых строк таблицы
def view(table_name):
    query = f"""
                SELECT * FROM {table_name}
                LIMIT 5
             """
    table = pd.io.sql.read_sql(query, con = engine)
    return table

<i>**Книги**

In [ ]:
view('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица содержит следующие столбцы:
- book_id ;
- author_id ;
- title ;
- num_pages ;
- publication_date ;
- publisher_id .

<i>**Авторы**

In [ ]:
view('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица содержит столбцы:
- author_id ;
- author .

<i>**Издательства**

In [ ]:
view('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица содержит столбцы:
- publisher_id ;
- publisher ;

<i>**Пользовательские оценки книг**

In [ ]:
view('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица содержит столбцы:
- rating_id ;
- book_id ;
- username ;
- rating .

<i>**Пользовательские обзоры**

In [ ]:
view('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица содержит столбцы:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

# <a name=2>Анализ данных.</a>

## <a name=2.1>Сколько книг вышло после 1 января 2000 года.</a>

In [ ]:
# Напишем функцию, выполняющую заданный запрос
def get_query(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

In [ ]:
get_query("""
            SELECT COUNT(book_id) AS books_count
            FROM books
            WHERE publication_date >= '2000-01-01'
          """)

,books_count
0,821


Мы получили количество книг, вышедших на сервисе после 1 января 2000 года - 821.

## <a name=2.2>Количество обзоров и средняя оценка книг.</a>

In [ ]:
get_query("""
              SELECT title, COUNT(DISTINCT review_id) as n_reviews, ROUND(AVG(rating), 2) as avg_rating
              FROM books
              LEFT JOIN reviews
                  ON reviews.book_id = books.book_id
              LEFT JOIN ratings
                  ON ratings.book_id = books.book_id
              GROUP BY books.book_id
              ORDER BY n_reviews DESC,
                       avg_rating DESC
          """)

,title,n_reviews,avg_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Book Thief,6,4.26
4,The Glass Castle,6,4.21
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Essential Tales and Poems,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Мы получили таблицу, содержащую информацию о количестве обзоров и среднем рейтинге всех книг на сервисе. Больше всего обзоров на книгу "Twilight" ("Сумерки", 1 книга из серии) - 7 обзоров, однако у нее не самый высокий рейтинг - всего 3,66. Далее идут 3 и 2 книги из серии "Гарри Поттер" - по 6 обзоров, средний рейтинг - 4,41 и 4,29 соответственно. На последнем месте книга "The Natural Way to Draw" - на нее не опубликовано ни одного обзора, а ее средний рейтинг 3,0.

## <a name=2.3>Издательство, выпустившее наибольшее число книг.</a>

Рассматриваем издательства, выпустившие наибольшее количество книг толщиной более 50 страниц, чтобы исключить брошюры

In [ ]:
get_query("""
              SELECT publisher, COUNT(book_id) AS books_count
              FROM publishers
              LEFT JOIN books
                  ON books.publisher_id = publishers.publisher_id
              WHERE num_pages > 50
              GROUP BY publishers.publisher_id
              ORDER BY COUNT(book_id) DESC
              LIMIT 1
          """)

,publisher,books_count
0,Penguin Books,42


Видим, что издательство, выпустившее наибольшее количество книг толщиной более 50 страниц, называется "Penguin Books".

## <a name=2.4>Автор с самой высокой средней оценкой книг.</a>

Будем учитывать только книги с 50 и более оценками

In [ ]:
get_query("""
              SELECT author, AVG(avg_rating) AS avg_author_rating
              FROM (
                  SELECT MAX(author) AS author, title, ROUND(AVG(rating), 2) AS avg_rating
                  FROM authors
                  LEFT JOIN books
                      ON authors.author_id = books.author_id
                  LEFT JOIN ratings
                      ON ratings.book_id = books.book_id
                  GROUP BY books.book_id
                  HAVING COUNT(DISTINCT rating_id) >= 50
              ) AS ar
              GROUP BY author
              ORDER BY avg_author_rating DESC
              LIMIT 1
          """)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.285


Дж. К. Роулинг и М. Грандпре. - соавторы, имеющие самую высокую среднюю оценку книг с 50 и более оценками (4,285)

## <a name=2.5>Среднее количество обзоров от пользователей, поставивших более 50 оценок.</a>

In [ ]:
get_query("""
              SELECT ROUND(AVG(reviews_count), 1) AS avg_reviews
              FROM (
                      SELECT COUNT(DISTINCT review_id) AS reviews_count
                      FROM ratings
                      LEFT JOIN reviews
                          ON ratings.username = reviews.username
                      GROUP BY ratings.username
                      HAVING COUNT(DISTINCT rating_id) > 50
                    ) AS rc
          """)

,avg_reviews
0,24.3


Получили среднее количество обзоров от пользователей, поставивших более 50 оценок - 24.

# <a name=3>Выводы.</a>

После 1 января 2000 года вышла 821 книга из 1000 имеющихся на сервисе.

Самое большое количество обзоров на первую книгу "Сумерки" (7 обзоров и средний рейтинг 3,66), "Гарри Поттер и Узник Азкабана" (6 обзоров и средний рейтинг 4,41) и "Гарри Поттер и Тайная Комната" (6 обзоров и средний рейтинг 4,29). На книгу "The Natural Way to Draw" не опубликовано ни одного обзора, а ее средний рейтинг 3,0 - самый низкий среди книг без обзоров.

Издательство "Penguin Books" выпустило наибольшее количество книг толщиной более 50 страниц (42 книги).

Соавторы, имеющие самую высокую пользовательскую среднюю оценку (среди книг, имеющих больше 50 оценок) - Дж. К. Роулинг и М. Грандпре (средний рейтинг - 4,285).

Среднее число обзоров книг для самых активных пользователей (которые поставили более 50 оценок) - 24.